<a href="https://colab.research.google.com/github/bforsbe/SK2534/blob/main/mutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mutate DNA sequence but preserve protein fold**
this exercise is meant to affirm the basics of how DNA is *translated* into proteins, and how this determines the *fold* of the protein.

### **Instructions**
The following is a set of code blocks that will set up and execute steps to help you re-design the DNA sequence of a small protein. **The aim is to attain a high mutation score**, which you will achieve by mutating DNA bases. Each mutation increases your score. If the mutation results in a mutated amino acid as well, your score will increase even more. **However, if the mutations destroy the fold of the protein, your total score will suffer**. Your total score is calculated by copying the mutated protein sequence into the ESMfold code block and predicting its structure. The following block can then calculate a similarity score in the form of RMSD, which is combined with your mutation score.

#### **What to do**
- read the instructions for each block.
- click the little ▶ play icon to the left of each block below.


First, we will install biopython, which is needed to translate DNA sequences, align structures, and more. this should be fairly quick.

In [4]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.8 MB/s eta 0:00:00


Ok, now we can provide an original and mutated sequence, check what is different, and get a mutation score. In case you need it, the unmutated DNA sequence we are using in this exercise is

```
ATG AAA GTG ATT TTT CTG AAA AAA GGC GAA ATT AAA AAC GTG GCG GAT GGC TAT GCG AAC AAC TTT CTG TTT AAA CAG GGC CTG GCG ATT GAA GCG
```

Try altering one of the bases, and run this block to see what happened, and what score you get.


In [7]:
#@title Mutate DNA and inspect translation
from IPython.display import display, HTML
from textwrap import wrap
import Bio
from Bio.Seq import Seq

# === Inputs ===
original_seq = "ATG AAA GTG ATT TTT CTG AAA AAA GGC GAA ATT AAA AAC GTG GCG GAT GGC TAT GCG AAC AAC TTT CTG TTT AAA CAG GGC CTG GCG ATT GAA GCG" #@param {type:"string"}
mutated_seq  = "ATG AAA GTG ATT TTT CTG AAA ATA GGC GAA ATT AAA AAC GTG GCG GAT GGC TAT GCG AAC AAC TTT CTG TTT AAA CAG GGC CTG GCG ATT GAA GCG" #@param {type:"string"}

# Clean input
original_seq = original_seq.replace(" ", "").upper()
mutated_seq  = mutated_seq.replace(" ", "").upper()

# === Checks ===
if len(original_seq) != len(mutated_seq):
    raise ValueError(f"Sequences must have the same length (got {len(original_seq)} vs {len(mutated_seq)}).")
if len(original_seq) % 3 != 0:
    raise ValueError(f"Sequence length must be a multiple of 3 (got {len(original_seq)}).")

# === Genetic code (DNA codons) ===
codon_table = {
    "TTT":"F","TTC":"F","TTA":"L","TTG":"L",
    "CTT":"L","CTC":"L","CTA":"L","CTG":"L",
    "ATT":"I","ATC":"I","ATA":"I","ATG":"M",
    "GTT":"V","GTC":"V","GTA":"V","GTG":"V",
    "TCT":"S","TCC":"S","TCA":"S","TCG":"S",
    "CCT":"P","CCC":"P","CCA":"P","CCG":"P",
    "ACT":"T","ACC":"T","ACA":"T","ACG":"T",
    "GCT":"A","GCC":"A","GCA":"A","GCG":"A",
    "TAT":"Y","TAC":"Y","TAA":"*","TAG":"*",
    "CAT":"H","CAC":"H","CAA":"Q","CAG":"Q",
    "AAT":"N","AAC":"N","AAA":"K","AAG":"K",
    "GAT":"D","GAC":"D","GAA":"E","GAG":"E",
    "TGT":"C","TGC":"C","TGA":"*","TGG":"W",
    "CGT":"R","CGC":"R","CGA":"R","CGG":"R",
    "AGT":"S","AGC":"S","AGA":"R","AGG":"R",
    "GGT":"G","GGC":"G","GGA":"G","GGG":"G"
}

def translate_dna(seq):
    codons = wrap(seq, 3)
    aa = [codon_table.get(c,"?") for c in codons]
    return codons, aa

from IPython.display import display, HTML

def compare_sequences(seq1, seq2, space_every=10, spacer=" ", no_number=False):
    """
    Display two sequences in monospace, highlight differences in bold red,
    insert spaces every n characters, and add numbering above each block.
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be the same length")

    def highlight_block(s1, s2):
        """Highlight differences in one block of equal length"""
        out1, out2 = [], []
        for a, b in zip(s1, s2):
            out1.append(a)
            if a == b:
                out2.append(b)
            else:
                out2.append(f"<span style='color:red;font-weight:bold'>{b}</span>")
        return "".join(out1), "".join(out2)

    # Split into blocks of size space_every
    blocks1 = [seq1[i:i+space_every] for i in range(0, len(seq1), space_every)]
    blocks2 = [seq2[i:i+space_every] for i in range(0, len(seq2), space_every)]

    seq1_blocks, seq2_blocks, num_blocks = [], [], []
    pos = 0
    for b1, b2 in zip(blocks1, blocks2):
        o1, o2 = highlight_block(b1, b2)
        seq1_blocks.append(o1)
        seq2_blocks.append(o2)
        pos += len(b1)
        num_blocks.append(str(pos).rjust(len(b1)))


    number_line = spacer.join(num_blocks)
    seq1_line   = spacer.join(seq1_blocks)
    seq2_line   = spacer.join(seq2_blocks)

    html = []
    if no_number:
        html = seq1_line + "<br>" + seq2_line
    else:
        html = number_line + "<br>" + seq1_line + "<br>" + seq2_line
    display(HTML(f"<pre style='font-family:monospace'>{html}</pre>"))

def dna_identity(seq1: str, seq2: str) -> float:
    """Return % identity at DNA level"""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")
    matches = sum(a == b for a, b in zip(seq1.upper(), seq2.upper()))
    return 100 * matches / len(seq1)

def protein_identity(seq1: str, seq2: str) -> float:
    """Translate DNA → protein and return % identity"""
    if len(seq1) % 3 != 0 or len(seq2) % 3 != 0:
        raise ValueError("DNA sequences must be divisible by 3")
    prot1 = str(Seq(seq1).translate(to_stop=False))
    prot2 = str(Seq(seq2).translate(to_stop=False))
    if len(prot1) != len(prot2):
        raise ValueError("Translated proteins have different lengths")
    matches = sum(a == b for a, b in zip(prot1, prot2))
    return 100 * matches / len(prot1)

def mutation_score(seq1: str, seq2: str) -> float:
    """Return mutation score"""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")

    protein_id = protein_identity(seq1, seq2)
    dna_id = dna_identity(seq1, seq2)

    return ((100.-protein_id)+(100.-dna_id))/2

# === Translation ===
codons_o, aa_o = translate_dna(original_seq)
codons_m, aa_m = translate_dna(mutated_seq)

# === HTML display ===

compare_sequences(original_seq,mutated_seq,space_every=3)
compare_sequences(aa_o,aa_m,space_every=1, no_number=True, spacer="   ")

#display(HTML(f"<pre style='font-family:monospace'>{"   ".join(aa_o)}</pre>"))
#compare_sequences("".join(codons_o),"".join(codons_m),space_every=30)

print("\n\nMutaded sequence without spaces:")
display(HTML(f"<pre style='font-family:monospace'>{"".join(aa_m)}</pre>"))

dna_id = dna_identity(original_seq, mutated_seq)
#print(f"\nDNA % identity: {dna_id:.1f}%")
protein_id = protein_identity(original_seq, mutated_seq)
#print(f"Protein % identity: {protein_id:.1f}%")
print("\n")
mut_score = mutation_score(original_seq,mutated_seq)
print(f"Mutation score is {mut_score:.2f}")




Mutaded sequence without spaces:




Mutation score is 2.08


#**ESMFold**
Once we have a mutated *protein* sequence, we can fold it to make sure we didn't destroy the fold. **NOTE** if your mutation did not destroy the fold, then your total score is the same as your mutation score (so you could try to maximize this without mutating protein at all). To fold the protein, we will be using ESMfold, a protein language model made by meta.

For more details on ESMfold see: [Github](https://github.com/facebookresearch/esm/tree/main/esm), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1)

#### **Instructions**
- Run the block to install ESMfold. This should only take a couple of minutes.
- Copy the "Mutaded sequence without spaces" from the preceding block above, into the "sequence" filed below.
- Run the prediction

**NOTE**: The prediction might take a minute the first time and be quicker when you run it subsequently. The speed might also depend on the resources you have in the session, which you can check in the upper corener of the colab notebook header.

In [4]:
%%time
#@title install
#@markdown install ESMFold, OpenFold and download Params (~2min 30s)
version = "1"
model_name = "esmfold_v0.model" if version == "0" else "esmfold.model"
import os, time
if not os.path.isfile(model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &")

  if not os.path.isfile("finished_install"):
    # install libs
    print("installing libs...")
    os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
    os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

    print("installing openfold...")
    # install openfold
    os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")

    print("installing esmfold...")
    # install esmfold
    os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")
    os.system("touch finished_install")

  # wait for Params to finish downloading...
  while not os.path.isfile(model_name):
    time.sleep(5)
  if os.path.isfile(f"{model_name}.aria2"):
    print("downloading params...")
  while os.path.isfile(f"{model_name}.aria2"):
    time.sleep(5)

installing libs...
installing openfold...
installing esmfold...
CPU times: user 6.55 ms, sys: 10.5 ms, total: 17 ms
Wall time: 3min 20s


In [23]:
#@title ##run **ESMFold**
%%time
from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np
import torch
from jax.tree_util import tree_map
import matplotlib.pyplot as plt
from scipy.special import softmax
import gc

def parse_output(output):
  pae = (output["aligned_confidence_probs"][0] * np.arange(64)).mean(-1) * 31
  plddt = output["plddt"][0,:,1]

  bins = np.append(0,np.linspace(2.3125,21.6875,63))
  sm_contacts = softmax(output["distogram_logits"],-1)[0]
  sm_contacts = sm_contacts[...,bins<8].sum(-1)
  xyz = output["positions"][-1,0,:,1]
  mask = output["atom37_atom_exists"][0,:,1] == 1
  o = {"pae":pae[mask,:][:,mask],
       "plddt":plddt[mask],
       "sm_contacts":sm_contacts[mask,:][:,mask],
       "xyz":xyz[mask]}
  return o

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)

jobname = "mutation_1" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]

sequence = "MKPIFLKWWESKNVADWYAWNFLFKQGLAIEA" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)
copies = 1
if copies == "" or copies <= 0: copies = 1
sequence = ":".join([sequence] * copies)
num_recycles = 3
chain_linker = 25

ID = jobname+"_"+get_hash(sequence)[:5]
print(ID)
seqs = sequence.split(":")
lengths = [len(s) for s in seqs]
length = sum(lengths)
print("length",length)

u_seqs = list(set(seqs))
if len(seqs) == 1: mode = "mono"
elif len(u_seqs) == 1: mode = "homo"
else: mode = "hetero"

if "model" not in dir() or model_name != model_name_:
  if "model" in dir():
    # delete old model from memory
    del model
    gc.collect()
    if torch.cuda.is_available():
      torch.cuda.empty_cache()

  model = torch.load(model_name, weights_only=False)
  model.eval().cuda().requires_grad_(False)
  model_name_ = model_name

# optimized for Tesla T4
if length > 700:
  model.set_chunk_size(64)
else:
  model.set_chunk_size(128)

torch.cuda.empty_cache()
output = model.infer(sequence,
                     num_recycles=num_recycles,
                     chain_linker="X"*chain_linker,
                     residue_index_offset=512)

pdb_str = model.output_to_pdb(output)[0]
output = tree_map(lambda x: x.cpu().numpy(), output)
ptm = output["ptm"][0]
plddt = output["plddt"][0,...,1].mean()
O = parse_output(output)
print(f'ptm: {ptm:.3f} plddt: {plddt:.3f}')
os.system(f"mkdir -p {ID}")
prefix = f"{ID}/ptm{ptm:.3f}_r{num_recycles}_default"
np.savetxt(f"{prefix}.pae.txt",O["pae"],"%.3f")
o_pdb_file=f"{prefix}.pdb"
with open(o_pdb_file,"w") as out:
  out.write(pdb_str)

mutation_1_dc2cf
length 32
ptm: 0.255 plddt: 54.518
CPU times: user 1.47 s, sys: 689 µs, total: 1.47 s
Wall time: 1.48 s


Now that we have predicted the fold of the mutated protein sequence, we will align it to a reference, and calculate their differences in atom positions, or root mean swuared difference (RMSD). This will then be used to penalize proteins with a high RMSD to the reference structure, which will affect your total score. That is, if you destroyed the fold, your score will be lowered.

**NOTE**: This will be based on the last calculated mutation score and predicted model, i.e. assuming that you have run all blocks in order (you dont have to run the install blocks again though, unless you have restarted the whole session).


In [24]:
#@title Calculate RMSD-based final score
import os, re, requests
import torch
from jax.tree_util import tree_map
from Bio.Seq import Seq
from Bio.PDB import PDBParser, Superimposer
import numpy as np

# --- Superpose prediction onto reference ---
def align_pred_to_ref(pred_pdb_str, ref_pdb_file, out_file="pred_aligned.pdb"):
    parser = PDBParser(QUIET=True)

    # Save predicted PDB string to temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdb") as tmp_pred:
        tmp_pred.write(pred_pdb_str.encode())
        pred_file = tmp_pred.name

    pred = parser.get_structure("pred", pred_file)
    ref  = parser.get_structure("ref", ref_pdb_file)

    # Use CA atoms for alignment
    pred_atoms = [a for a in pred.get_atoms() if a.get_id() == "CA"]
    ref_atoms  = [a for a in ref.get_atoms()  if a.get_id() == "CA"]

    n = min(len(pred_atoms), len(ref_atoms))
    si = Superimposer()
    si.set_atoms(ref_atoms[:n], pred_atoms[:n])  # align prediction to reference
    si.apply(pred.get_atoms())                   # apply transform to prediction

    # Write aligned prediction
    io = PDBIO()
    io.set_structure(pred)
    io.save(out_file)

    return out_file, si.rms

# --- Function for RMSD ---
def structure_rmsd(pdb1: str, pdb2: str) -> float:
    parser = PDBParser(QUIET=True)
    s1 = parser.get_structure("ref", pdb1)
    s2 = parser.get_structure("pred", pdb2)

    atoms1, atoms2 = [], []
    for a1, a2 in zip(s1.get_atoms(), s2.get_atoms()):
        if a1.get_id() in ["N", "CA", "C"]:  # backbone only
            atoms1.append(a1)
            atoms2.append(a2)

    if len(atoms1) != len(atoms2):
        raise ValueError("Structures must have same number of backbone atoms")

    sup = Superimposer()
    sup.set_atoms(atoms1, atoms2)
    return sup.rms

def rmsd_to_similarity(rmsd, sigma=2.0):
    """
    Convert RMSD (Å) to a similarity score in [0,1].
    Smaller RMSD -> score close to 1, larger RMSD -> score close to 0.

    Parameters
    ----------
    rmsd : float or np.array
        Root-mean-square deviation in angstroms.
    sigma : float
        Scaling factor controlling decay. Typical RMSDs of 1-2 Å → high similarity.

    Returns
    -------
    similarity : float or np.array
        Score in [0,1].
    """
    return np.exp(-rmsd / sigma)

# --- Download reference PDB ---
ref_file="reference.pdb"
!wget -q -O reference.pdb https://raw.githubusercontent.com/bforsbe/cryonet25/main/ptm0.638_r3_default.pdb

aligned_file, rmsd = align_pred_to_ref(pdb_str, ref_file)

#for rmsd in [0.5, 1.0, 2.0, 5.0]:
#    print(rmsd, rmsd_to_similarity(rmsd))

# --- Compare ---
#o_pdb_file="test_71f75/ptm0.645_r3_default.pdb"
print(f"input file is {o_pdb_file}")
#protein_seq = "MKVIFLKKGEIKNVADGYANNFLFKQGLAIEA
rmsd_val = rmsd #structure_rmsd(ref_file, o_pdb_file)
#print(f"Protein sequence: {protein_seq}")
print(f"RMSD vs reference: {rmsd_val:.3f} Å (score: {rmsd_to_similarity(rmsd_val):.2f})")
print("\n\n")
print(f"Total score: {mutation_score(original_seq,mutated_seq)*rmsd_to_similarity(rmsd_val):.3f}")


input file is mutation_1_dc2cf/ptm0.255_r3_default.pdb
RMSD vs reference: 7.921 Å (score: 0.02)



Total score: 0.169


To understand what the RMSD value means, you can choose to look at the reference and prediction aligned in 3D. For RMSD values below 2, they might superpose very closely, whereas you might see two completely different folds for higher values.

In [1]:
#@title Display predicted (superposed) + reference {run: "auto"}
import py3Dmol
from Bio.PDB import PDBParser, Superimposer, PDBIO
import tempfile, os

# --- Visualize both ---
def show_pred_and_ref_aligned(pred_pdb_str, ref_pdb_file,
                              color="pLDDT", show_sidechains=False, show_mainchains=False,
                              chains=None, vmin=50, vmax=90, size=(800,480), Ls=None):

    aligned_file, rmsd = align_pred_to_ref(pred_pdb_str, ref_pdb_file)

    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',
                        width=size[0], height=size[1])

    # Reference (fixed, green)
    with open(ref_pdb_file) as f:
        ref_pdb_str = f.read()
    view.addModel(ref_pdb_str,'pdb')
    view.setStyle({'model':0},{'cartoon':{'color':'green','opacity':0.7}})

    # Prediction (aligned, colored)
    with open(aligned_file) as f:
        pred_aligned_str = f.read()
    view.addModel(pred_aligned_str,'pdb')
    if color == "pLDDT":
        view.setStyle({'model':1,'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
    elif color == "rainbow":
        view.setStyle({'model':1,'cartoon': {'color':'spectrum'}})
    elif color == "chain":
        from string import ascii_uppercase
        alphabet_list = list(ascii_uppercase)
        for n,chain,c in zip(range(chains),alphabet_list,pymol_color_list):
            view.setStyle({'model':1,'chain':chain},{'cartoon': {'color':c}})

    view.zoomTo()
    print(f"Structural alignment RMSD: {rmsd:.3f} Å")
    return view


# --- Params ---
color = "confidence" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = True #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

show_pred_and_ref_aligned(pdb_str, "reference.pdb",
                          color=color,
                          show_sidechains=show_sidechains,
                          show_mainchains=show_mainchains,
                          Ls=lengths).show()


ModuleNotFoundError: No module named 'py3Dmol'

The following is only useful if you want to assess the prediction, which is not the topic here, but something you can do and discuss with the instructor if you want.

In [2]:
#@title plot confidence (optional)

dpi = 100 #@param {type:"integer"}

def plot_ticks(Ls):
  Ln = sum(Ls)
  L_prev = 0
  for L_i in Ls[:-1]:
    L = L_prev + L_i
    L_prev += L_i
    plt.plot([0,Ln],[L,L],color="black")
    plt.plot([L,L],[0,Ln],color="black")
  ticks = np.cumsum([0]+Ls)
  ticks = (ticks[1:] + ticks[:-1])/2
  plt.yticks(ticks,alphabet_list[:len(ticks)])

def plot_confidence(O, Ls=None, dpi=100):
  if "lm_contacts" in O:
    plt.figure(figsize=(20,4), dpi=dpi)
    plt.subplot(1,4,1)
  else:
    plt.figure(figsize=(15,4), dpi=dpi)
    plt.subplot(1,3,1)

  plt.title('Predicted lDDT')
  plt.plot(O["plddt"])
  if Ls is not None:
    L_prev = 0
    for L_i in Ls[:-1]:
      L = L_prev + L_i
      L_prev += L_i
      plt.plot([L,L],[0,100],color="black")
  plt.xlim(0,O["plddt"].shape[0])
  plt.ylim(0,100)
  plt.ylabel('plDDT')
  plt.xlabel('position')
  plt.subplot(1,4 if "lm_contacts" in O else 3,2)

  plt.title('Predicted Aligned Error')
  Ln = O["pae"].shape[0]
  plt.imshow(O["pae"],cmap="bwr",vmin=0,vmax=30,extent=(0, Ln, Ln, 0))
  if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
  plt.colorbar()
  plt.xlabel('Scored residue')
  plt.ylabel('Aligned residue')

  if "lm_contacts" in O:
    plt.subplot(1,4,3)
    plt.title("contacts from LM")
    plt.imshow(O["lm_contacts"],cmap="Greys",vmin=0,vmax=1,extent=(0, Ln, Ln, 0))
    if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
    plt.subplot(1,4,4)
  else:
    plt.subplot(1,3,3)
  plt.title("contacts from Structure Module")
  plt.imshow(O["sm_contacts"],cmap="Greys",vmin=0,vmax=1,extent=(0, Ln, Ln, 0))
  if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
  return plt

plot_confidence(O, Ls=lengths, dpi=dpi)
plt.savefig(f'{prefix}.png',bbox_inches='tight')
plt.show()

NameError: name 'O' is not defined

If you also want to download the prediction to look at in pymol or any other viewer on your local computer, you can do that as well.

In [ ]:
#@title download predictions
from google.colab import files
os.system(f"zip {ID}.zip {ID}/*")
files.download(f'{ID}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>